# create-input-data-v1-0

Read in the co reports and create features for modelling.

In [87]:
import pandas as pd
from datetime import datetime
from time import strftime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests, json, pickle
import nltk, re

import tika
from tika import parser

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# nltk.download('stopwords')
# nltk.download('wordnet')

In [6]:
# Initialise variables and objects

tika.initVM()
ps = PorterStemmer()
lmtzr = WordNetLemmatizer()

### Read in tickers and dates sheet¶

In [3]:
tickers = pd.read_csv('~/Downloads/report-dates - Master.csv', parse_dates=['AR Date', 'FY end', 'Period Start Date', 'Period End Date'])

tickers['Symbol Yahoo'] = tickers['Symbol'].astype(str) + ".L"

tickers.shape
tickers
tickers.dtypes

(54, 9)

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,AR link,Symbol Yahoo
0,ABCAM,ABC,2016-11-09,2016-06-30,73,2016-09-25,2017-08-28,https://uploads.abcamplc.com/wp-content/upload...,ABC.L
1,ADVANCED MEDICAL SOLUTIONS,AMS,2017-03-14,2016-12-31,73,2017-03-28,2018-02-28,https://www.admedsol.com/wp-content/uploads/20...,AMS.L
2,ALLIANCE PHARMA,APH,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,http://investors.alliancepharmaceuticals.com/~...,APH.L
3,ASOS,ASC,2016-10-28,2016-08-31,58,2016-11-11,2017-10-14,https://www.asosplc.com/~/media/Files/A/Asos-V...,ASC.L
4,BLUE PRISM,PRSM,2017-01-24,2016-10-31,85,2017-07-02,2018-10-01,https://www.blueprism.com/wp-content/uploads/2...,PRSM.L
5,boohoo.com,BOO,2016-05-23,2016-02-29,84,2016-06-06,2017-09-05,http://www.boohooplc.com/~/media/Files/B/Booho...,BOO.L
6,BROOKS MACDONALD GROUP,BRK,2016-09-21,2016-06-30,83,2016-05-10,2017-07-09,https://www.brooksmacdonald.com/~/media/Files/...,BRK.L
7,BURFORD CAPITAL,BUR,2017-03-13,2016-12-31,72,2017-03-27,2018-02-27,http://www.burfordcapital.com/wp-content/uploa...,BUR.L
8,CAMELLIA,CAM,2017-02-04,2016-12-31,92,2017-04-16,2018-03-19,https://www.camellia.plc.uk/pages/8a7c0775-d4e...,CAM.L
9,CARETECH HLDG,CTH,2017-07-02,2016-09-30,130,2017-02-21,2018-01-24,https://www.caretech-uk.com/~/media/Files/C/Ca...,CTH.L


Company                      object
Symbol                       object
AR Date              datetime64[ns]
FY end               datetime64[ns]
YE to Report lag              int64
Period Start Date    datetime64[ns]
Period End Date      datetime64[ns]
AR link                      object
Symbol Yahoo                 object
dtype: object

In [4]:
# Get AR report URL as list

url_list = (tickers['AR link'].tolist())
    
for i, item in enumerate(url_list):
    print(i, item)

0 https://uploads.abcamplc.com/wp-content/uploads/2016/11/13165533/Abcam2016AR.pdf
1 https://www.admedsol.com/wp-content/uploads/2015/03/2016-Annual-Report.pdf
2 http://investors.alliancepharmaceuticals.com/~/media/Files/A/Alliance-Pharma-IR/documents/alliancepharma-ara-2016.pdf
3 https://www.asosplc.com/~/media/Files/A/Asos-V2/2016-in-review/financial-review/28-10-2016-ar.pdf
4 https://www.blueprism.com/wp-content/uploads/2017/03/Blue-Prism-Full-Year-Results-presentation-24.01.17.pdf
5 http://www.boohooplc.com/~/media/Files/B/Boohoo/reports-and-presentations/ar-2016.pdf?pdfdata=1
6 https://www.brooksmacdonald.com/~/media/Files/B/Brooks-Macdonald-V3/documents/shareholder-services/fy-results-2016-presentation.pdf
7 http://www.burfordcapital.com/wp-content/uploads/2017/03/BUR-26890-Annual-Report-2016-web.pdf
8 https://www.camellia.plc.uk/pages/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4/articles/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4.pdf
9 https://www.caretech-uk.com/~/media/Files/C/Caretech-UK/r

### Create a dataset which has one element per source AR document

In [21]:
# Loop through url_list

text_cleaned_all = []

for i, url in enumerate(url_list):
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "   Processing document no. ", i, " : ", url)
    parsed = parser.from_file(url, 'http://localhost:9998/tika')
    prep = re.sub('[^a-zA-Z]', ' ', parsed["content"]).lower().split()
    lemmatized = [lmtzr.lemmatize(word) for word in prep if not word in set(stopwords.words('english'))]
    lemmatized = ' '.join(lemmatized)
    text_cleaned_all.append(lemmatized)
    
len(text_cleaned_all)   # Should be one row per AR url

2018-08-16 13:14:42,046 [MainThread  ] [INFO ]  Retrieving https://uploads.abcamplc.com/wp-content/uploads/2016/11/13165533/Abcam2016AR.pdf to /tmp/https-uploads-abcamplc-com-wp-content-uploads-2016-11-13165533-abcam2016ar-pdf.


2018-08-16 13:14:42    Processing document no.  0  :  https://uploads.abcamplc.com/wp-content/uploads/2016/11/13165533/Abcam2016AR.pdf


2018-08-16 13:28:52,725 [MainThread  ] [INFO ]  Retrieving https://www.admedsol.com/wp-content/uploads/2015/03/2016-Annual-Report.pdf to /tmp/https-www-admedsol-com-wp-content-uploads-2015-03-2016-annual-report-pdf.


2018-08-16 13:28:52    Processing document no.  1  :  https://www.admedsol.com/wp-content/uploads/2015/03/2016-Annual-Report.pdf


HTTPError: HTTP Error 403: Forbidden

In [25]:
# Create as function to run one URL at a time
def clean_ar_text(url):
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "   Processing document: ", url)
    parsed = parser.from_file(url, 'http://localhost:9998/tika')
    prep = re.sub('[^a-zA-Z]', ' ', parsed["content"]).lower().split()
    lemmatized = [lmtzr.lemmatize(word) for word in prep if not word in set(stopwords.words('english'))]
    lemmatized = ' '.join(lemmatized)
    
    print("Cleaned text length: ", len(lemmatized))
    return lemmatized

In [26]:
ar_cleaned_abcom = clean_ar_text("https://uploads.abcamplc.com/wp-content/uploads/2016/11/13165533/Abcam2016AR.pdf")


2018-08-16 13:56:20,794 [MainThread  ] [INFO ]  Retrieving https://uploads.abcamplc.com/wp-content/uploads/2016/11/13165533/Abcam2016AR.pdf to /tmp/https-uploads-abcamplc-com-wp-content-uploads-2016-11-13165533-abcam2016ar-pdf.


2018-08-16 13:56:20    Processing document:  https://uploads.abcamplc.com/wp-content/uploads/2016/11/13165533/Abcam2016AR.pdf
Cleaned text length:  264204


In [28]:
ar_cleaned_admedsol = clean_ar_text("/home/stuart/Downloads/admedsol-2016-Annual-Report.pdf")


2018-08-16 15:16:20    Processing document:  /home/stuart/Downloads/admedsol-2016-Annual-Report.pdf
Cleaned text length:  220840


In [29]:
ar_cleaned_2 = clean_ar_text("http://investors.alliancepharmaceuticals.com/~/media/Files/A/Alliance-Pharma-IR/documents/alliancepharma-ara-2016.pdf")

2018-08-16 15:44:00,964 [MainThread  ] [INFO ]  Retrieving http://investors.alliancepharmaceuticals.com/~/media/Files/A/Alliance-Pharma-IR/documents/alliancepharma-ara-2016.pdf to /tmp/http-investors-alliancepharmaceuticals-com-media-files-a-alliance-pharma-ir-documents-alliancepharma-ara-2016-pdf.


2018-08-16 15:44:00    Processing document:  http://investors.alliancepharmaceuticals.com/~/media/Files/A/Alliance-Pharma-IR/documents/alliancepharma-ara-2016.pdf
Cleaned text length:  145484


In [31]:
ar_cleaned_3 = clean_ar_text("https://www.asosplc.com/~/media/Files/A/Asos-V2/2016-in-review/financial-review/28-10-2016-ar.pdf")

2018-08-16 16:25:59,146 [MainThread  ] [INFO ]  Retrieving https://www.asosplc.com/~/media/Files/A/Asos-V2/2016-in-review/financial-review/28-10-2016-ar.pdf to /tmp/https-www-asosplc-com-media-files-a-asos-v2-2016-in-review-financial-review-28-10-2016-ar-pdf.


2018-08-16 16:25:59    Processing document:  https://www.asosplc.com/~/media/Files/A/Asos-V2/2016-in-review/financial-review/28-10-2016-ar.pdf
Cleaned text length:  231223


In [32]:
ar_cleaned_4 = clean_ar_text("https://www.blueprism.com/wp-content/uploads/2017/03/Blue-Prism-Full-Year-Results-presentation-24.01.17.pdf")

2018-08-16 16:43:57,697 [MainThread  ] [INFO ]  Retrieving https://www.blueprism.com/wp-content/uploads/2017/03/Blue-Prism-Full-Year-Results-presentation-24.01.17.pdf to /tmp/https-www-blueprism-com-wp-content-uploads-2017-03-blue-prism-full-year-results-presentation-24-01-17-pdf.


2018-08-16 16:43:57    Processing document:  https://www.blueprism.com/wp-content/uploads/2017/03/Blue-Prism-Full-Year-Results-presentation-24.01.17.pdf
Cleaned text length:  15174


In [33]:
ar_cleaned_5 = clean_ar_text("http://www.boohooplc.com/~/media/Files/B/Boohoo/reports-and-presentations/ar-2016.pdf?pdfdata=1")
ar_cleaned_6 = clean_ar_text("https://www.brooksmacdonald.com/~/media/Files/B/Brooks-Macdonald-V3/documents/shareholder-services/fy-results-2016-presentation.pdf")
ar_cleaned_7 = clean_ar_text("http://www.burfordcapital.com/wp-content/uploads/2017/03/BUR-26890-Annual-Report-2016-web.pdf")


2018-08-16 16:51:02,113 [MainThread  ] [INFO ]  Retrieving http://www.boohooplc.com/~/media/Files/B/Boohoo/reports-and-presentations/ar-2016.pdf?pdfdata=1 to /tmp/http-www-boohooplc-com-media-files-b-boohoo-reports-and-presentations-ar-2016-pdf-pdfdata-1.


2018-08-16 16:51:02    Processing document:  http://www.boohooplc.com/~/media/Files/B/Boohoo/reports-and-presentations/ar-2016.pdf?pdfdata=1


2018-08-16 16:55:54,836 [MainThread  ] [INFO ]  Retrieving https://www.brooksmacdonald.com/~/media/Files/B/Brooks-Macdonald-V3/documents/shareholder-services/fy-results-2016-presentation.pdf to /tmp/https-www-brooksmacdonald-com-media-files-b-brooks-macdonald-v3-documents-shareholder-services-fy-results-2016-presentation-pdf.


Cleaned text length:  129181
2018-08-16 16:55:54    Processing document:  https://www.brooksmacdonald.com/~/media/Files/B/Brooks-Macdonald-V3/documents/shareholder-services/fy-results-2016-presentation.pdf


2018-08-16 16:56:11,771 [MainThread  ] [INFO ]  Retrieving http://www.burfordcapital.com/wp-content/uploads/2017/03/BUR-26890-Annual-Report-2016-web.pdf to /tmp/http-www-burfordcapital-com-wp-content-uploads-2017-03-bur-26890-annual-report-2016-web-pdf.


Cleaned text length:  11377
2018-08-16 16:56:11    Processing document:  http://www.burfordcapital.com/wp-content/uploads/2017/03/BUR-26890-Annual-Report-2016-web.pdf


2018-08-16 16:59:27,774 [MainThread  ] [INFO ]  Retrieving https://www.camellia.plc.uk/pages/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4/articles/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4.pdf to /tmp/https-www-camellia-plc-uk-pages-8a7c0775-d4ea-483c-98c3-1abbcb98a9f4-articles-8a7c0775-d4ea-483c-98c3-1abbcb98a9f4-pdf.


Cleaned text length:  128773
2018-08-16 16:59:27    Processing document:  https://www.camellia.plc.uk/pages/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4/articles/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4.pdf


URLError: <urlopen error [Errno 110] Connection timed out>

In [36]:
ar_cleaned_9 = clean_ar_text("https://www.caretech-uk.com/~/media/Files/C/Caretech-UK/reports-and-presentations/annual-report-2016.pdf")


2018-08-16 17:25:54,665 [MainThread  ] [INFO ]  Retrieving https://www.caretech-uk.com/~/media/Files/C/Caretech-UK/reports-and-presentations/annual-report-2016.pdf to /tmp/https-www-caretech-uk-com-media-files-c-caretech-uk-reports-and-presentations-annual-report-2016-pdf.


2018-08-16 17:25:54    Processing document:  https://www.caretech-uk.com/~/media/Files/C/Caretech-UK/reports-and-presentations/annual-report-2016.pdf


2018-08-16 17:30:44,128 [MainThread  ] [INFO ]  Retrieving https://www.centralasiametals.com/wp-content/uploads/2016/05/CAML-2016-Annual-Report-FINAL.pdf to /tmp/https-www-centralasiametals-com-wp-content-uploads-2016-05-caml-2016-annual-report-final-pdf.


Cleaned text length:  169299
2018-08-16 17:30:44    Processing document:  https://www.centralasiametals.com/wp-content/uploads/2016/05/CAML-2016-Annual-Report-FINAL.pdf


HTTPError: HTTP Error 403: Forbidden

In [40]:
ar_cleaned_10 = clean_ar_text("/home/stuart/Downloads/CAML-2016-Annual-Report-FINAL.pdf")

2018-08-16 17:42:14    Processing document:  /home/stuart/Downloads/CAML-2016-Annual-Report-FINAL.pdf
Cleaned text length:  159469


In [42]:
ar_cleaned_11 = clean_ar_text("/home/stuart/Downloads/25520_clinigen_ar2016.pdf")

2018-08-16 17:49:10    Processing document:  /home/stuart/Downloads/25520_clinigen_ar2016.pdf
Cleaned text length:  226206


In [43]:
ar_cleaned_12 = clean_ar_text("https://www.cvsukltd.co.uk/wp-content/uploads/2016/09/cvs-group-plc-annual-report-2016-3.pdf")
ar_cleaned_13 = clean_ar_text("https://www.dartgroup.co.uk/uploadedFiles/Dart_Group_Plc/Shared_Resources_(Images,_docs_etc)/Documents/Report_And_Accounts/Report-and-Accounts-2016.pdf")
ar_cleaned_14 = clean_ar_text("https://www.emisgroupplc.com/media/1360/emis-group-plc-annual-report-and-accounts-2016.pdf")


2018-08-16 18:06:57,435 [MainThread  ] [INFO ]  Retrieving https://www.cvsukltd.co.uk/wp-content/uploads/2016/09/cvs-group-plc-annual-report-2016-3.pdf to /tmp/https-www-cvsukltd-co-uk-wp-content-uploads-2016-09-cvs-group-plc-annual-report-2016-3-pdf.


2018-08-16 18:06:57    Processing document:  https://www.cvsukltd.co.uk/wp-content/uploads/2016/09/cvs-group-plc-annual-report-2016-3.pdf


2018-08-16 18:10:46,579 [MainThread  ] [INFO ]  Retrieving https://www.dartgroup.co.uk/uploadedFiles/Dart_Group_Plc/Shared_Resources_(Images,_docs_etc)/Documents/Report_And_Accounts/Report-and-Accounts-2016.pdf to /tmp/https-www-dartgroup-co-uk-uploadedfiles-dart_group_plc-shared_resources_-images-_docs_etc-documents-report_and_accounts-report-and-accounts-2016-pdf.


Cleaned text length:  136291
2018-08-16 18:10:46    Processing document:  https://www.dartgroup.co.uk/uploadedFiles/Dart_Group_Plc/Shared_Resources_(Images,_docs_etc)/Documents/Report_And_Accounts/Report-and-Accounts-2016.pdf


2018-08-16 18:14:33,861 [MainThread  ] [INFO ]  Retrieving https://www.emisgroupplc.com/media/1360/emis-group-plc-annual-report-and-accounts-2016.pdf to /tmp/https-www-emisgroupplc-com-media-1360-emis-group-plc-annual-report-and-accounts-2016-pdf.


Cleaned text length:  130179
2018-08-16 18:14:33    Processing document:  https://www.emisgroupplc.com/media/1360/emis-group-plc-annual-report-and-accounts-2016.pdf
Cleaned text length:  241953


In [44]:
ar_cleaned_15 = clean_ar_text("http://www.fp.fo/wp-content/uploads/2017/05/fp-annual-report-2016-4234njjb42m3.pdf")
ar_cleaned_16 = clean_ar_text("http://fevertree1.d3r-cdn.com/pdfs/original/3094-fevertree-ar-2017-web-ready.pdf")
ar_cleaned_17 = clean_ar_text("https://www.firstderivatives.com/wp-content/uploads/2016/08/Accounts_2016_First_Derivatives_PLC.pdf")


2018-08-16 18:25:03,914 [MainThread  ] [INFO ]  Retrieving http://www.fp.fo/wp-content/uploads/2017/05/fp-annual-report-2016-4234njjb42m3.pdf to /tmp/http-www-fp-fo-wp-content-uploads-2017-05-fp-annual-report-2016-4234njjb42m3-pdf.


2018-08-16 18:25:03    Processing document:  http://www.fp.fo/wp-content/uploads/2017/05/fp-annual-report-2016-4234njjb42m3.pdf


2018-08-16 18:31:20,542 [MainThread  ] [INFO ]  Retrieving http://fevertree1.d3r-cdn.com/pdfs/original/3094-fevertree-ar-2017-web-ready.pdf to /tmp/http-fevertree1-d3r-cdn-com-pdfs-original-3094-fevertree-ar-2017-web-ready-pdf.


Cleaned text length:  221828
2018-08-16 18:31:20    Processing document:  http://fevertree1.d3r-cdn.com/pdfs/original/3094-fevertree-ar-2017-web-ready.pdf


2018-08-16 18:35:18,791 [MainThread  ] [INFO ]  Retrieving https://www.firstderivatives.com/wp-content/uploads/2016/08/Accounts_2016_First_Derivatives_PLC.pdf to /tmp/https-www-firstderivatives-com-wp-content-uploads-2016-08-accounts_2016_first_derivatives_plc-pdf.


Cleaned text length:  130873
2018-08-16 18:35:18    Processing document:  https://www.firstderivatives.com/wp-content/uploads/2016/08/Accounts_2016_First_Derivatives_PLC.pdf
Cleaned text length:  154177


In [45]:
ar_cleaned_18 = clean_ar_text("https://frontier-drupal.s3-eu-west-1.amazonaws.com/development/frontier-corp/s3fs-public/press-releases/financial/Frontier-AR-Full-2016.pdf")



2018-08-16 18:42:00,902 [MainThread  ] [INFO ]  Retrieving https://frontier-drupal.s3-eu-west-1.amazonaws.com/development/frontier-corp/s3fs-public/press-releases/financial/Frontier-AR-Full-2016.pdf to /tmp/https-frontier-drupal-s3-eu-west-1-amazonaws-com-development-frontier-corp-s3fs-public-press-releases-financial-frontier-ar-full-2016-pdf.


2018-08-16 18:42:00    Processing document:  https://frontier-drupal.s3-eu-west-1.amazonaws.com/development/frontier-corp/s3fs-public/press-releases/financial/Frontier-AR-Full-2016.pdf


2018-08-16 18:45:36,564 [MainThread  ] [INFO ]  Retrieving https://www.gamma.co.uk/wp-content/uploads/2017/04/gamma-annual-report-2016.pdf to /tmp/https-www-gamma-co-uk-wp-content-uploads-2017-04-gamma-annual-report-2016-pdf.


Cleaned text length:  123036
2018-08-16 18:45:36    Processing document:  https://www.gamma.co.uk/wp-content/uploads/2017/04/gamma-annual-report-2016.pdf


HTTPError: HTTP Error 403: Forbidden

In [47]:
ar_cleaned_19 = clean_ar_text("/home/stuart/Downloads/gamma-annual-report-2016.pdf")
ar_cleaned_20 = clean_ar_text("https://www.gbgplc.com/media/2307/gbg-annual-report-2016.pdf")

2018-08-16 19:30:29    Processing document:  /home/stuart/Downloads/gamma-annual-report-2016.pdf


2018-08-16 19:35:52,029 [MainThread  ] [INFO ]  Retrieving https://www.gbgplc.com/media/2307/gbg-annual-report-2016.pdf to /tmp/https-www-gbgplc-com-media-2307-gbg-annual-report-2016-pdf.


Cleaned text length:  191932
2018-08-16 19:35:52    Processing document:  https://www.gbgplc.com/media/2307/gbg-annual-report-2016.pdf
Cleaned text length:  179354


In [ ]:
# # Do remaining ones in a loop

# text_cleaned_all = []
# exceptions = []

# counter = 21

# for url in enumerate(url_list[counter:]):
#     try:
#         print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "   Processing document no. ", counter, " : ", url)
#         parsed = parser.from_file(url, 'http://localhost:9998/tika')
#         prep = re.sub('[^a-zA-Z]', ' ', parsed["content"]).lower().split()
#         lemmatized = [lmtzr.lemmatize(word) for word in prep if not word in set(stopwords.words('english'))]
#         lemmatized = ' '.join(lemmatized)
#         row_output = [counter, lemmatized]
#         text_cleaned_all.append(row_output)
#         counter += 1
#     except:
#         row_output = [counter, url]
#         exceptions.append(row_output)
#         counter += 1
    
# print("Length of cleaned output: ", len(text_cleaned_all))   # Should be one row per AR url
# print("Length of excpetions: ", len(exceptions))   # Should be one row per AR url

In [49]:
ar_cleaned_21 = clean_ar_text("https://goochandhousego.com/wp-content/uploads/2016/12/7-Dec-16-GH-2016-Annual-Report.pdf")
ar_cleaned_22 = clean_ar_text("https://www.hurricaneenergy.com/~/media/Files/H/Hurricane-Energy/reports/Annual-report2016.pdf")
ar_cleaned_23 = clean_ar_text("https://www.chi-med.com/wp-content/uploads/2017/03/2016AR-Form20F.pdf")


2018-08-16 19:42:36,539 [MainThread  ] [INFO ]  Retrieving https://goochandhousego.com/wp-content/uploads/2016/12/7-Dec-16-GH-2016-Annual-Report.pdf to /tmp/https-goochandhousego-com-wp-content-uploads-2016-12-7-dec-16-gh-2016-annual-report-pdf.


2018-08-16 19:42:36    Processing document:  https://goochandhousego.com/wp-content/uploads/2016/12/7-Dec-16-GH-2016-Annual-Report.pdf


2018-08-16 19:47:02,703 [MainThread  ] [INFO ]  Retrieving https://www.hurricaneenergy.com/~/media/Files/H/Hurricane-Energy/reports/Annual-report2016.pdf to /tmp/https-www-hurricaneenergy-com-media-files-h-hurricane-energy-reports-annual-report2016-pdf.


Cleaned text length:  149983
2018-08-16 19:47:02    Processing document:  https://www.hurricaneenergy.com/~/media/Files/H/Hurricane-Energy/reports/Annual-report2016.pdf


2018-08-16 19:50:26,408 [MainThread  ] [INFO ]  Retrieving https://www.chi-med.com/wp-content/uploads/2017/03/2016AR-Form20F.pdf to /tmp/https-www-chi-med-com-wp-content-uploads-2017-03-2016ar-form20f-pdf.


Cleaned text length:  113575
2018-08-16 19:50:26    Processing document:  https://www.chi-med.com/wp-content/uploads/2017/03/2016AR-Form20F.pdf
Cleaned text length:  959570


In [50]:
ar_cleaned_24 = clean_ar_text("https://www.iomart.com/wp-content/uploads/2016/06/iomart-annual-accs_2016.pdf")
ar_cleaned_25 = clean_ar_text("http://www.iqep.com/media/149511/2016AnnualReportWEB.pdf")
ar_cleaned_26 = clean_ar_text("http://cfrenzy.webfactional.com/pdfs/reportsandaccounts/2016/JH%20plc%20Report%20and%20Accounts%20June%202016.pdf")


2018-08-16 20:30:50,446 [MainThread  ] [INFO ]  Retrieving https://www.iomart.com/wp-content/uploads/2016/06/iomart-annual-accs_2016.pdf to /tmp/https-www-iomart-com-wp-content-uploads-2016-06-iomart-annual-accs_2016-pdf.


2018-08-16 20:30:50    Processing document:  https://www.iomart.com/wp-content/uploads/2016/06/iomart-annual-accs_2016.pdf


2018-08-16 20:36:05,292 [MainThread  ] [INFO ]  Retrieving http://www.iqep.com/media/149511/2016AnnualReportWEB.pdf to /tmp/http-www-iqep-com-media-149511-2016annualreportweb-pdf.


Cleaned text length:  176366
2018-08-16 20:36:05    Processing document:  http://www.iqep.com/media/149511/2016AnnualReportWEB.pdf


2018-08-16 20:42:12,600 [MainThread  ] [INFO ]  Retrieving http://cfrenzy.webfactional.com/pdfs/reportsandaccounts/2016/JH%20plc%20Report%20and%20Accounts%20June%202016.pdf to /tmp/http-cfrenzy-webfactional-com-pdfs-reportsandaccounts-2016-jh-20plc-20report-20and-20accounts-20june-202016-pdf.


Cleaned text length:  199137
2018-08-16 20:42:12    Processing document:  http://cfrenzy.webfactional.com/pdfs/reportsandaccounts/2016/JH%20plc%20Report%20and%20Accounts%20June%202016.pdf
Cleaned text length:  93187


In [51]:
ar_cleaned_27 = clean_ar_text("http://www.jsg.com/media/2249444/annual-report-2016.pdf")
ar_cleaned_28 = clean_ar_text("https://www.keywordsstudios.com/content/uploads/2017/07/Keywords-Studios-Annual-Report-2016.pdf")
ar_cleaned_29 = clean_ar_text("https://ltgplc.com/wp-content/uploads/2017/04/LTG_Report_2016.pdf")


2018-08-16 21:02:18,022 [MainThread  ] [INFO ]  Retrieving http://www.jsg.com/media/2249444/annual-report-2016.pdf to /tmp/http-www-jsg-com-media-2249444-annual-report-2016-pdf.


2018-08-16 21:02:18    Processing document:  http://www.jsg.com/media/2249444/annual-report-2016.pdf


2018-08-16 21:12:41,870 [MainThread  ] [INFO ]  Retrieving https://www.keywordsstudios.com/content/uploads/2017/07/Keywords-Studios-Annual-Report-2016.pdf to /tmp/https-www-keywordsstudios-com-content-uploads-2017-07-keywords-studios-annual-report-2016-pdf.


Cleaned text length:  337135
2018-08-16 21:12:41    Processing document:  https://www.keywordsstudios.com/content/uploads/2017/07/Keywords-Studios-Annual-Report-2016.pdf


2018-08-16 21:17:05,931 [MainThread  ] [INFO ]  Retrieving https://ltgplc.com/wp-content/uploads/2017/04/LTG_Report_2016.pdf to /tmp/https-ltgplc-com-wp-content-uploads-2017-04-ltg_report_2016-pdf.


Cleaned text length:  148232
2018-08-16 21:17:05    Processing document:  https://ltgplc.com/wp-content/uploads/2017/04/LTG_Report_2016.pdf
Cleaned text length:  136858


In [54]:
ar_cleaned_30 = clean_ar_text("/home/stuart/Downloads/2017-04-13+Annual+Report+2016.pdf")
ar_cleaned_31 = clean_ar_text("/home/stuart/Downloads/2016 Annual Report.pdf")


2018-08-16 21:29:08    Processing document:  /home/stuart/Downloads/2017-04-13+Annual+Report+2016.pdf
Cleaned text length:  151035
2018-08-16 21:33:48    Processing document:  /home/stuart/Downloads/2016 Annual Report.pdf
Cleaned text length:  113176


In [55]:
ar_cleaned_32 = clean_ar_text("https://majesticwineplc.co.uk/wp-content/uploads/Annual-Report-Final-2016.pdf")


2018-08-16 21:37:19,595 [MainThread  ] [INFO ]  Retrieving https://majesticwineplc.co.uk/wp-content/uploads/Annual-Report-Final-2016.pdf to /tmp/https-majesticwineplc-co-uk-wp-content-uploads-annual-report-final-2016-pdf.


2018-08-16 21:37:19    Processing document:  https://majesticwineplc.co.uk/wp-content/uploads/Annual-Report-Final-2016.pdf
Cleaned text length:  168062


In [56]:
ar_cleaned_33 = clean_ar_text("http://www.midwichgroupplc.com/media/1148/midwich-ar2016.pdf")
ar_cleaned_34 = clean_ar_text("https://www.mulberry.com/plugins/investor_relations/pdf/annual_report_2016.pdf")
ar_cleaned_35 = clean_ar_text("https://cdn.next15.com/uploads/2017/09/2016-Annual-Report.pdf")


2018-08-16 21:42:32,288 [MainThread  ] [INFO ]  Retrieving http://www.midwichgroupplc.com/media/1148/midwich-ar2016.pdf to /tmp/http-www-midwichgroupplc-com-media-1148-midwich-ar2016-pdf.


2018-08-16 21:42:32    Processing document:  http://www.midwichgroupplc.com/media/1148/midwich-ar2016.pdf


2018-08-16 21:46:57,041 [MainThread  ] [INFO ]  Retrieving https://www.mulberry.com/plugins/investor_relations/pdf/annual_report_2016.pdf to /tmp/https-www-mulberry-com-plugins-investor_relations-pdf-annual_report_2016-pdf.


Cleaned text length:  152532
2018-08-16 21:46:57    Processing document:  https://www.mulberry.com/plugins/investor_relations/pdf/annual_report_2016.pdf


2018-08-16 21:50:26,923 [MainThread  ] [INFO ]  Retrieving https://cdn.next15.com/uploads/2017/09/2016-Annual-Report.pdf to /tmp/https-cdn-next15-com-uploads-2017-09-2016-annual-report-pdf.


Cleaned text length:  116902
2018-08-16 21:50:26    Processing document:  https://cdn.next15.com/uploads/2017/09/2016-Annual-Report.pdf
Cleaned text length:  171033


In [57]:
ar_cleaned_36 = clean_ar_text("https://www.nicholsplc.co.uk/Content/ReportPDFs/2016/2016%20Annual%20Report%20Web.pdf")
ar_cleaned_37 = clean_ar_text("http://www.numiscorp.com/financialreports/NU3367%202016%20Annual_Combined_FINAL.pdf")
ar_cleaned_38 = clean_ar_text("http://investors.patisserieholdings.co.uk/~/media/Files/P/Patisserie-Valerie-IR/results-presentations/patisserie-holdings-year-ended-300916.pdf")


2018-08-16 21:55:26,448 [MainThread  ] [INFO ]  Retrieving https://www.nicholsplc.co.uk/Content/ReportPDFs/2016/2016%20Annual%20Report%20Web.pdf to /tmp/https-www-nicholsplc-co-uk-content-reportpdfs-2016-2016-20annual-20report-20web-pdf.


2018-08-16 21:55:26    Processing document:  https://www.nicholsplc.co.uk/Content/ReportPDFs/2016/2016%20Annual%20Report%20Web.pdf


2018-08-16 21:58:51,960 [MainThread  ] [INFO ]  Retrieving http://www.numiscorp.com/financialreports/NU3367%202016%20Annual_Combined_FINAL.pdf to /tmp/http-www-numiscorp-com-financialreports-nu3367-202016-20annual_combined_final-pdf.


Cleaned text length:  102717
2018-08-16 21:58:51    Processing document:  http://www.numiscorp.com/financialreports/NU3367%202016%20Annual_Combined_FINAL.pdf


2018-08-16 22:03:48,086 [MainThread  ] [INFO ]  Retrieving http://investors.patisserieholdings.co.uk/~/media/Files/P/Patisserie-Valerie-IR/results-presentations/patisserie-holdings-year-ended-300916.pdf to /tmp/http-investors-patisserieholdings-co-uk-media-files-p-patisserie-valerie-ir-results-presentations-patisserie-holdings-year-ended-300916-pdf.


Cleaned text length:  171672
2018-08-16 22:03:48    Processing document:  http://investors.patisserieholdings.co.uk/~/media/Files/P/Patisserie-Valerie-IR/results-presentations/patisserie-holdings-year-ended-300916.pdf
Cleaned text length:  76969


In [58]:
ar_cleaned_39 = clean_ar_text("http://www.polarcapital.co.uk/ResourceModule.aspx/Pdf/PolarCapital-AnnualReportAndAccounts-Corporate-enGB-31.03.16.pdf?key=7710239e-821e-4b69-8bab-b4d124d0e934")
ar_cleaned_40 = clean_ar_text("https://pbinvestmedia.s3.amazonaws.com/uploads/presentation/media_url/62/Purplebricks-Group-plc-AR-2016.pdf")


2018-08-16 22:06:21,807 [MainThread  ] [INFO ]  Retrieving http://www.polarcapital.co.uk/ResourceModule.aspx/Pdf/PolarCapital-AnnualReportAndAccounts-Corporate-enGB-31.03.16.pdf?key=7710239e-821e-4b69-8bab-b4d124d0e934 to /tmp/http-www-polarcapital-co-uk-resourcemodule-aspx-pdf-polarcapital-annualreportandaccounts-corporate-engb-31-03-16-pdf-key-7710239e-821e-4b69-8bab-b4d124d0e934.


2018-08-16 22:06:21    Processing document:  http://www.polarcapital.co.uk/ResourceModule.aspx/Pdf/PolarCapital-AnnualReportAndAccounts-Corporate-enGB-31.03.16.pdf?key=7710239e-821e-4b69-8bab-b4d124d0e934


2018-08-16 22:09:45,429 [MainThread  ] [INFO ]  Retrieving https://pbinvestmedia.s3.amazonaws.com/uploads/presentation/media_url/62/Purplebricks-Group-plc-AR-2016.pdf to /tmp/https-pbinvestmedia-s3-amazonaws-com-uploads-presentation-media_url-62-purplebricks-group-plc-ar-2016-pdf.


Cleaned text length:  121150
2018-08-16 22:09:45    Processing document:  https://pbinvestmedia.s3.amazonaws.com/uploads/presentation/media_url/62/Purplebricks-Group-plc-AR-2016.pdf


2018-08-16 22:11:56,662 [MainThread  ] [INFO ]  Retrieving https://www.redde.com/wp-content/uploads/2018/03/Annual-Report-and-Accounts-09-2016.pdf to /tmp/https-www-redde-com-wp-content-uploads-2018-03-annual-report-and-accounts-09-2016-pdf.


Cleaned text length:  81390
2018-08-16 22:11:56    Processing document:  https://www.redde.com/wp-content/uploads/2018/03/Annual-Report-and-Accounts-09-2016.pdf


HTTPError: HTTP Error 403: Forbidden

In [59]:
ar_cleaned_41 = clean_ar_text("/home/stuart/Downloads/Annual-Report-and-Accounts-09-2016.pdf")

2018-08-16 22:20:07    Processing document:  /home/stuart/Downloads/Annual-Report-and-Accounts-09-2016.pdf
Cleaned text length:  128719


In [60]:
ar_cleaned_42 = clean_ar_text("https://www.renewholdings.com/files/5814/8230/3424/Renew_Holdings_plcAnnual_Report_and_Accounts_2016.pdf")


2018-08-16 22:24:00,622 [MainThread  ] [INFO ]  Retrieving https://www.renewholdings.com/files/5814/8230/3424/Renew_Holdings_plcAnnual_Report_and_Accounts_2016.pdf to /tmp/https-www-renewholdings-com-files-5814-8230-3424-renew_holdings_plcannual_report_and_accounts_2016-pdf.


2018-08-16 22:24:00    Processing document:  https://www.renewholdings.com/files/5814/8230/3424/Renew_Holdings_plcAnnual_Report_and_Accounts_2016.pdf


2018-08-16 22:28:26,730 [MainThread  ] [INFO ]  Retrieving https://www.rws.com/wp-content/uploads/2017/09/rws_holdings_annual_report_2016.pdf to /tmp/https-www-rws-com-wp-content-uploads-2017-09-rws_holdings_annual_report_2016-pdf.


Cleaned text length:  163844
2018-08-16 22:28:26    Processing document:  https://www.rws.com/wp-content/uploads/2017/09/rws_holdings_annual_report_2016.pdf


HTTPError: HTTP Error 403: Forbidden

In [62]:
ar_cleaned_43 = clean_ar_text("/home/stuart/Downloads/rws_holdings_annual_report_2016.pdf")
ar_cleaned_44 = clean_ar_text("http://www.scapa.com/files/Scapa_RA_2016.pdf")


2018-08-16 22:32:01    Processing document:  /home/stuart/Downloads/rws_holdings_annual_report_2016.pdf


2018-08-16 22:34:50,224 [MainThread  ] [INFO ]  Retrieving http://www.scapa.com/files/Scapa_RA_2016.pdf to /tmp/http-www-scapa-com-files-scapa_ra_2016-pdf.


Cleaned text length:  100063
2018-08-16 22:34:50    Processing document:  http://www.scapa.com/files/Scapa_RA_2016.pdf
Cleaned text length:  272983


In [63]:
ar_cleaned_45 = clean_ar_text("http://secureincomereit.co.uk/wp-content/uploads/2017/03/Secure-Income-REIT-December-2016-FINAL-GLOSSY.pdf")
ar_cleaned_46 = clean_ar_text("https://www.sms-plc.com/media/1086/smart-metering-systems-plc-annual-report-and-accounts-2016.pdf")
ar_cleaned_47 = clean_ar_text("https://www.soundenergyplc.com/media/1487/sound-energy-annual-report-2016-web.pdf")


2018-08-17 07:59:59,585 [MainThread  ] [INFO ]  Retrieving http://secureincomereit.co.uk/wp-content/uploads/2017/03/Secure-Income-REIT-December-2016-FINAL-GLOSSY.pdf to /tmp/http-secureincomereit-co-uk-wp-content-uploads-2017-03-secure-income-reit-december-2016-final-glossy-pdf.


2018-08-17 07:59:59    Processing document:  http://secureincomereit.co.uk/wp-content/uploads/2017/03/Secure-Income-REIT-December-2016-FINAL-GLOSSY.pdf


2018-08-17 08:02:43,018 [MainThread  ] [INFO ]  Retrieving https://www.sms-plc.com/media/1086/smart-metering-systems-plc-annual-report-and-accounts-2016.pdf to /tmp/https-www-sms-plc-com-media-1086-smart-metering-systems-plc-annual-report-and-accounts-2016-pdf.


Cleaned text length:  161407
2018-08-17 08:02:43    Processing document:  https://www.sms-plc.com/media/1086/smart-metering-systems-plc-annual-report-and-accounts-2016.pdf


2018-08-17 08:05:00,745 [MainThread  ] [INFO ]  Retrieving https://www.soundenergyplc.com/media/1487/sound-energy-annual-report-2016-web.pdf to /tmp/https-www-soundenergyplc-com-media-1487-sound-energy-annual-report-2016-web-pdf.


Cleaned text length:  136598
2018-08-17 08:05:00    Processing document:  https://www.soundenergyplc.com/media/1487/sound-energy-annual-report-2016-web.pdf
Cleaned text length:  162575


In [64]:
ar_cleaned_48 = clean_ar_text("/home/stuart/Downloads/Staffline-Group-Annual-Report-2016-online.pdf")
ar_cleaned_49 = clean_ar_text("https://www.telfordhomes-ir.london/media/1020/annual-report-march2016.pdf")
ar_cleaned_50 = clean_ar_text("http://www.fwthorpe.co.uk/pdf/fwthorpe_final_16.pdf")


2018-08-17 08:22:08    Processing document:  /home/stuart/Downloads/Staffline-Group-Annual-Report-2016-online.pdf


2018-08-17 08:25:40,595 [MainThread  ] [INFO ]  Retrieving https://www.telfordhomes-ir.london/media/1020/annual-report-march2016.pdf to /tmp/https-www-telfordhomes-ir-london-media-1020-annual-report-march2016-pdf.


Cleaned text length:  156116
2018-08-17 08:25:40    Processing document:  https://www.telfordhomes-ir.london/media/1020/annual-report-march2016.pdf


2018-08-17 08:29:23,105 [MainThread  ] [INFO ]  Retrieving http://www.fwthorpe.co.uk/pdf/fwthorpe_final_16.pdf to /tmp/http-www-fwthorpe-co-uk-pdf-fwthorpe_final_16-pdf.


Cleaned text length:  134444
2018-08-17 08:29:23    Processing document:  http://www.fwthorpe.co.uk/pdf/fwthorpe_final_16.pdf
Cleaned text length:  152440


In [65]:
ar_cleaned_51 = clean_ar_text("http://www.watkinjonesplc.com/~/media/Files/W/Watkin-Jones/documents/watkin-jones-plc-annual-report-2016-v1.pdf")
ar_cleaned_52 = clean_ar_text("http://www.youngs.co.uk/data/investors/results/50_resultsDocument.pdf")
ar_cleaned_53 = clean_ar_text("http://www.youngs.co.uk/data/investors/results/50_resultsDocument.pdf")

2018-08-17 08:35:01,441 [MainThread  ] [INFO ]  Retrieving http://www.watkinjonesplc.com/~/media/Files/W/Watkin-Jones/documents/watkin-jones-plc-annual-report-2016-v1.pdf to /tmp/http-www-watkinjonesplc-com-media-files-w-watkin-jones-documents-watkin-jones-plc-annual-report-2016-v1-pdf.


2018-08-17 08:35:01    Processing document:  http://www.watkinjonesplc.com/~/media/Files/W/Watkin-Jones/documents/watkin-jones-plc-annual-report-2016-v1.pdf


2018-08-17 08:39:13,658 [MainThread  ] [INFO ]  Retrieving http://www.youngs.co.uk/data/investors/results/50_resultsDocument.pdf to /tmp/http-www-youngs-co-uk-data-investors-results-50_resultsdocument-pdf.


Cleaned text length:  160585
2018-08-17 08:39:13    Processing document:  http://www.youngs.co.uk/data/investors/results/50_resultsDocument.pdf


2018-08-17 08:43:30,468 [MainThread  ] [INFO ]  Retrieving http://www.youngs.co.uk/data/investors/results/50_resultsDocument.pdf to /tmp/http-www-youngs-co-uk-data-investors-results-50_resultsdocument-pdf.


Cleaned text length:  153522
2018-08-17 08:43:30    Processing document:  http://www.youngs.co.uk/data/investors/results/50_resultsDocument.pdf
Cleaned text length:  153522


In [66]:
ar_cleaned_8 = clean_ar_text("https://www.camellia.plc.uk/pages/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4/articles/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4.pdf")


2018-08-17 09:55:35,029 [MainThread  ] [INFO ]  Retrieving https://www.camellia.plc.uk/pages/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4/articles/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4.pdf to /tmp/https-www-camellia-plc-uk-pages-8a7c0775-d4ea-483c-98c3-1abbcb98a9f4-articles-8a7c0775-d4ea-483c-98c3-1abbcb98a9f4-pdf.


2018-08-17 09:55:35    Processing document:  https://www.camellia.plc.uk/pages/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4/articles/8a7c0775-d4ea-483c-98c3-1abbcb98a9f4.pdf
Cleaned text length:  151991


In [84]:
# Stack all outputs together

text_cleaned_all = []

text_cleaned_all = [ar_cleaned_abcom, ar_cleaned_admedsol, ar_cleaned_2, ar_cleaned_3, ar_cleaned_4 \
                    , ar_cleaned_5, ar_cleaned_6, ar_cleaned_7, ar_cleaned_8, ar_cleaned_9 \
                    , ar_cleaned_10, ar_cleaned_11, ar_cleaned_12, ar_cleaned_13, ar_cleaned_14 \
                    , ar_cleaned_15, ar_cleaned_16, ar_cleaned_17, ar_cleaned_18, ar_cleaned_19 \
                    , ar_cleaned_20, ar_cleaned_21, ar_cleaned_22, ar_cleaned_23, ar_cleaned_24 \
                    , ar_cleaned_25, ar_cleaned_26, ar_cleaned_27, ar_cleaned_28, ar_cleaned_29 \
                    , ar_cleaned_30, ar_cleaned_31, ar_cleaned_32, ar_cleaned_33, ar_cleaned_34 \
                    , ar_cleaned_35, ar_cleaned_36, ar_cleaned_37, ar_cleaned_38, ar_cleaned_39 \
                    , ar_cleaned_40, ar_cleaned_41, ar_cleaned_42, ar_cleaned_43, ar_cleaned_44 \
                    , ar_cleaned_45, ar_cleaned_46, ar_cleaned_47, ar_cleaned_48, ar_cleaned_49 \
                    , ar_cleaned_50, ar_cleaned_51, ar_cleaned_52, ar_cleaned_53 ]

len(text_cleaned_all)
len(text_cleaned_all[0])

54

264204

In [88]:
# Save to permanent

with open('data/ar_text_cleaned.pkl', 'wb') as f:
    pickle.dump(text_cleaned_all, f)

In [ ]:
# # To load pickle back in later
# with open('data/ar_text_cleaned.pkl', 'rb') as f:
#     ar_text_cleaned = pickle.load(f)

In [90]:
# Save as CSV
pd.DataFrame(text_cleaned_all).to_csv('output/ar_text_cleaned.csv')

### Scrap code